In [1]:
import argparse
import os
from matplotlib import pyplot as plt

import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import log_loss
from decentralized_SGD_logistic import DecentralizedSGDLogistic
from sklearn.linear_model import SGDClassifier
from scipy.special import expit as sigmoid

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataset_path = os.path.expanduser('../ChocoSGD/data/rcv1_test.binary.bz2')
print('Loading dataset...')
A, y = load_svmlight_file(dataset_path)
A_p = A[:100000]
y_p = y[:100000]
#A_p = np.load('A_p.npy')
#y_p = np.load('y_p.npy')

Loading dataset...


In [4]:
def run_logistic(A, y, param):
    m = DecentralizedSGDLogistic(**param)
    res = m.fit(A, y)
    print('{} - score: {}'.format(param, m.score(A, y)))
    return res, m

In [5]:
clf = SGDClassifier(loss='log', penalty='l2',alpha =1/A_p.shape[0])
clf.fit(A_p, y_p)
print(clf.score(A_p, y_p))
x_predict = clf.predict(A_p)
exact_optimum = log_loss(x_predict, y_p) + (1 / A_p.shape[0]) * np.sum(x_predict**2)

/home/adirlou/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.97885


In [6]:
clf.coef_

array([[ 0.005435  , -0.36933332, -0.15702861, ...,  0.        ,
        -0.11969226,  0.00295827]])

In [7]:
exact_optimum

1.7305024850446937

In [8]:
y_p = 1*(y_p > 0)

In [17]:
reg = 1 / A_p.shape[0]
n_features = A_p.shape[1]

params_disconnected = dict(num_epoch=5, lr_type='bottou',
                           initial_lr=10, tau=n_features, regularizer=reg,
                           quantization='full', n_machines=128,
                           method='plain',topology='complete', estimate='final',
                           split_data_random_seed=2, distribute_data=True,
                           split_data_strategy='naive')

res_disconnected, dec_log = run_logistic(A_p, y_p, params_disconnected)

"""params_ring = dict(name="chocosgd-centralized", num_epoch=5, lr_type='bottou',
                           initial_lr=0.2, tau=n_features, regularizer=reg,
                           quantization='full', n_cores=10,
                           method='plain', topology='centralized', estimate='final',
                           split_data_random_seed=2, distribute_data=True,
                           split_data_strategy='naive')

res_centralized = run_logistic(A_p, y_p, params_ring)"""


length of indices: 2083
length of last machine indices: 48
current iteration: 0 epoch: 0 epoch_iteration 0 loss 0.9943298551341314 elapsed 7.30172324180603s
current iteration: 4 epoch: 0 epoch_iteration 4 loss 0.964589576757364 elapsed 16.31217074394226s
current iteration: 8 epoch: 0 epoch_iteration 8 loss 0.9432617264247996 elapsed 25.228818893432617s
current iteration: 12 epoch: 0 epoch_iteration 12 loss 0.9251992565064062 elapsed 40.04841589927673s
current iteration: 16 epoch: 0 epoch_iteration 16 loss 0.90983434100316 elapsed 48.9926552772522s
current iteration: 20 epoch: 0 epoch_iteration 20 loss 0.8970765178839832 elapsed 63.81248450279236s
current iteration: 24 epoch: 0 epoch_iteration 24 loss 0.8794143765275285 elapsed 72.84181046485901s


KeyboardInterrupt: 